# Mini-lab 3 - Geocoding, Maps, and Markers

In [2]:
from datascience import *
import folium
import numpy as np
from geopy.geocoders import Nominatim

### Geocoding
Geocoding is the process of converting locations (such as address or points of interest) into geographic coordinates (such as lat, lon coordinates). The geopy.geocoders package gives us access to several geolocation API's (like google maps, Nominatim - open street maps API, bing maps API, etc.). The documentations for the geocoding package can be found here: https://geopy.readthedocs.io/en/1.10.0/. 


#### Geocoding example
Below is an example of how the geocoding service works:

In [3]:
geolocator = Nominatim()
location = geolocator.geocode("175 5th Avenue NYC")
location


Location(Flatiron Building, 175, 5th Avenue, Flatiron District, Manhattan, Manhattan Community Board 5, New York County, NYC, New York, 10010, USA, (40.7410861, -73.9896298241625, 0.0))

#### location properties
The geolocator.geocode() returns a Location object. The location objects have the following properites:

In [4]:
print (location.latitude)
print (location.longitude)
print (location.altitude) 
print (location.address)

40.7410861
-73.9896298241625
0.0
Flatiron Building, 175, 5th Avenue, Flatiron District, Manhattan, Manhattan Community Board 5, New York County, NYC, New York, 10010, USA


#### Use the geolocater to find the lat, lon coordinates for the Statue of Liberty

In [5]:
liberty = geolocator.geocode("Statue of Liberty")
lib_lat = liberty.latitude
lib_lon = liberty.longitude
liberty

Location(Statue of Liberty, Flagpole Plaza, Manhattan Community Board 1, New York County, NYC, Liberty Island, 10004, USA, (40.6892474, -74.0445405280149, 0.0))

# Creating maps
If we generate a table with columns 'latitude', 'longitude', 'popup', and 'color', then we can generate maps with  markers from the table using the maps.Marker.map_table() method. See the example below. Click on the marker and the popup text should appear. 

In [6]:
location = geolocator.geocode("175 5th Avenue NYC")
table = Table().with_columns(['latitude', [location.latitude],
                              'longitude', [location.longitude],
                              'popup', [location.address],
                              'color', ['red']])
table

latitude,longitude,popup,color
40.7411,-73.9896,"Flatiron Building, 175, 5th Avenue, Flatiron District, M ...",red


In [7]:
maps.Marker.map_table(table)

/srv/app/venv/lib/python3.6/site-packages/datascience/maps.py:368: FutureWarning: simple_marker is deprecated. Use add_children(Marker) instead
  f(**self._folium_kwargs)


### Add a row to the map table for the Statue of Liberty. Make the color 'green'.
The [Table.append()](http://data8.org/datascience/_autosummary/datascience.tables.Table.append.html?highlight=append#datascience.tables.Table.append) method may come in handy for adding a row.

In [8]:
table = table.append([lib_lat, lib_lon, liberty.address, "green"])
table

latitude,longitude,popup,color
40.7411,-73.9896,"Flatiron Building, 175, 5th Avenue, Flatiron District, M ...",red
40.6892,-74.0445,"Statue of Liberty, Flagpole Plaza, Manhattan Community B ...",green


In [9]:
# after you have added the row, use the following to map the table
maps.Marker.map_table(table)

/srv/app/venv/lib/python3.6/site-packages/datascience/maps.py:368: FutureWarning: simple_marker is deprecated. Use add_children(Marker) instead
  f(**self._folium_kwargs)


### Variations in map style:
If we add a radius column to our table, we can use circle markers instead of info markers. See the example below

In [10]:
#Observe the way we construct the circular_marker. You will be constructing other markers in the same format.

#This line creates the base map, centered on 'location' and has a zoom level of 10.
map = folium.Map(location=[37.0, -122.0], zoom_start=10)

#This line creates a marker by passing the six arguments in it.
circular_marker = folium.CircleMarker(location=[37.0, -122.0],
                            radius = 600,
                            popup="My marker",
                            fill_color='blue',
                            color='grey',
                            fill_opacity=0.7)

#This line adds the new marker to the map
map.add_child(circular_marker)

### Mapping utilities and services in San Francisco
Mapping can be an important tool for visualizing spatial data. In the rest of this lab we will become more familiar with the mapping tools.

#### Firestations in SF
'firestations.csv' contains a list of addresses of firestations in San Francisco. 

In [11]:
firestations = Table.read_table('firestations.csv')
firestations

Station #,Address
Station 1,935 Folsom Street
Station 2,1340 Powell Street
Station 3,1067 Post Street
Station 4,449 Mission Rock Street
Station 5,1301 Turk Street
Station 6,135 Sanchez Street
Station 7,2300 Folsom Street
Station 8,36 Bluxome Street
Station 9,2245 Jerrold Avenue
Station 10,655 Presidio Avenue


**Your tasks:** 
1. Get the first address from the firestation table. Add ", San Francisco, CA" to this address.
1. Use the geolocater to find the lat, lon coordinates of this address.


In [14]:
first_add = firestations.column(1).item(0) + ", San Francisco, CA"
first_add_geo_lat = geolocator.geocode(first_add).latitude
first_add_geo_lon = geolocator.geocode(first_add).longitude
[first_add_geo_lat, first_add_geo_lon]

[37.77941665, -122.404100451486]

### Mapping all firestations
1. Use a **for loop** to iterate through all addresses in the dataframe. Add ", San Francisco, CA" to each address, then use the geolocater to find the lat, lon coordinates of the address,  
1. Append the latitude and longitude info to the firestations_map_table. Set the popup text = the Station # (from the firestation table), make the color "black" and the radius 200. 
1. Map the firestation_map_table using "maps.Circle.map_table()" method


In [15]:
#Your code here
#Creates an empty table with these headers
fire_stations_map_table = Table(['latitude', 'longitude', 'popup', 'color', 'radius'])

#creates a new base map
map = folium.Map(location=[37.75, -122.4], zoom_start=12)

#do all the things inside the for loop. You can also use 'for i in np.arange(firestations.nuw_rows)' and tweak the codes accordingly.
for address in firestations.column(1):
    
    #station is the name of the fire station(e.g. "Station 1")
    station = firestations.where(1, address).column(0).item(0)
    
    #new_loc is the location OBJECT of corresponding the firestation
    new_loc = geolocator.geocode([address + ", San Francisco, CA"])
    
    #add these information to the table
    fire_stations_map_table.append([new_loc.latitude, new_loc.longitude, station, "black", 200])
    
    #creates a marker for the station
    marker = folium.CircleMarker(location=[new_loc.latitude, new_loc.longitude],
                            radius = 200,
                            popup=station,
                            fill_color='black',
                            color='grey',
                            fill_opacity=0.7)
    #add the marker to the map
    map.add_child(marker)

In [16]:
map

### Bonus problem: There's a fire at City Hall!
If you want an extra challenge:

There's a fire at City Hall! We need to send firetrucks from the closest 3 firestations ASAP! Find City Hall, then find the 3 closest firestations and update your map by coloring these firestations red.
1. Add a large blue marker to the firestaion_map_table for the city hall.
1. Locate the closest 3 firestations to SF city hall 
1. Change the color of these firestation to red in your fire_stations_map_table.

In [17]:
#part 1: Find City Hall's location and add to the map
#You should be able to do this part like before
folium.CircleMarker(...)

In [18]:
#part 2: find the closest fire stations

#Optional: create a helper function
def distance_to_Hall(lat, lon):
    """Calculate the distance between a given location and City Hall"""
    return ...

#Use helper function on the table to get distances from fire stations to City Hall
distances = ...

#Take the three closest fire stations
closest_fire_stations = ...
closest_fire_stations

Ellipsis

In [31]:
#part 3: change them to red
#Hint: use another for loop to go through the three-row table and add each row as a marker to the map 
for i in np.arange(3):
    send_trucks = folium.CircleMarker(...)                                  
    map.add_child(send_trucks)
map